# DATA LOADING + DATA CLEANING

# Brief Summary of Our DuckDB Merging Process

## 1. GOAL

Merge all seven remote IMDb .tsv.gz datasets into one master Parquet file. The final table should contain all information for each title, joined across all relevant tables.

## 2. CHALLENGES

* Datasets are remote (hosted on Azure Blob Storage).

* Files are large and numerous; loading them all into memory (e.g., with Pandas) would crash the kernel.

* We need an efficient way to perform SQL-style joins without manual chunking.

## 3. APPROACH (DuckDB Virtual Tables & Staged Joins)

We use DuckDB to handle all the heavy lifting directly, without loading data into Python.

1. Connect: Create an in-memory DuckDB database (:memory:).

2. Register Files as VIEWs: Use CREATE OR REPLACE VIEW ... AS SELECT * FROM read_csv(...) for all seven .tsv.gz files.

   * DuckDB reads directly from the HTTPS URLs.

   * It decompresses the gzip files on the fly.

   * It handles TSV format (delim='\\t') and null values (nullstr='\\\\N').

   * VIEWs are lazy and cost no memory; they are just stored queries.

3. Staged Joins: Instead of one massive join, we build the final table in steps, materializing each result into a new TABLE. This controls memory usage.

   * merged_core: title.basics + ratings + crew + episode

   * merged_with_akas: merged_core + title.akas

   * merged_with_principals: merged_with_akas + title.principals

   * imdb_final: merged_with_principals + name.basics

4. Export to Parquet: Use the COPY command to stream the final imdb_final table directly to a compressed Parquet file (imdb_merged_duckdb.parquet).

## 4. HOW MERGING WORKS (SQL)

* We use LEFT JOIN ... USING (tconst) or LEFT JOIN ... ON (mp.nconst = n.nconst).

* LEFT JOIN: This is the equivalent of Pandas how="left". It keeps every row from the "left" table (e.g., merged_with_akas) and joins any matching information from the "right" table (e.g., title.principals). If there's no match, the columns from the right table are filled with NULL.

* USING (tconst): This is a convenient shortcut for ON table_A.tconst = table_B.tconst when the key column has the same name in both tables.

## 5. ADVANTAGES OF THE DUCKDB APPROACH

* Dedicated Compute: Running this pipeline on a capable Azure ML compute instance (like computeAlan) provides the necessary CPU and memory resources for DuckDB to perform these large-scale operations efficiently.

* Extremely Memory-Efficient: At no point is the entire dataset loaded into Python's memory. DuckDB streams data from the remote files, processes it, and streams the output to disk.

* Very Fast: DuckDB's vectorized query engine is highly optimized for this kind of work (joins, filtering, writing Parquet).

* No Manual Chunking: We don't need to write complex Python loops to manage chunks; DuckDB handles the streaming internally.

* Simple & Declarative: The logic is expressed in clean, standard SQL.

## 6. NOTES AFTER MERGE

* Multiple Rows per Title (Expected!): The final Parquet file will have multiple rows for a single tconst (title). This is normal and correct, as a title can have:

  * One row per actor/crew member (from title.principals).

  * One row per alternate title/region (from title.akas).

* Next Step (Cleaning): This merged file is now the perfect input for a cleaning pipeline (like the Dask script). During cleaning, you can:

  * Handle any NULL values that resulted from the joins.

  * Normalize genres, types, etc.

  * Potentially aggregate data (e.g., group by tconst and create a list of actors).

In [ ]:
import duckdb
import time
from tqdm import tqdm

# =======================
#  Azure URLs
# =======================
URL_NAME_BASICS    = "https://workspace4824871889.blob.core.windows.net/azureml-blobstore-84f516da-0fe5-4f33-8f3c-f18ec8e2b4f7/UI/2025-10-22_104122_UTC/name.basics.tsv.gz"
URL_TITLE_AKAS     = "https://workspace4824871889.blob.core.windows.net/azureml-blobstore-84f516da-0fe5-4f33-8f3c-f18ec8e2b4f7/UI/2025-10-22_104546_UTC/title.akas.tsv.gz"
URL_TITLE_BASICS   = "https://workspace4824871889.blob.core.windows.net/azureml-blobstore-84f516da-0fe5-4f33-8f3c-f18ec8e2b4f7/UI/2025-10-22_104810_UTC/title.basics.tsv.gz"
URL_TITLE_CREW     = "https://workspace4824871889.blob.core.windows.net/azureml-blobstore-84f516da-0fe5-4f33-8f3c-f18ec8e2b4f7/UI/2025-10-22_104937_UTC/title.crew.tsv.gz"
URL_TITLE_EPISODE  = "https://workspace4824871889.blob.core.windows.net/azureml-blobstore-84f516da-0fe5-4f33-8f3c-f18ec8e2b4f7/UI/2025-10-22_105103_UTC/title.episode.tsv.gz"
URL_TITLE_PRINC    = "https://workspace4824871889.blob.core.windows.net/azureml-blobstore-84f516da-0fe5-4f33-8f3c-f18ec8e2b4f7/UI/2025-10-22_105225_UTC/title.principals.tsv.gz"
URL_TITLE_RATINGS  = "https://workspace4824871889.blob.core.windows.net/azureml-blobstore-84f516da-0fe5-4f33-8f3c-f18ec8e2b4f7/UI/2025-10-22_105430_UTC/title.ratings.tsv.gz"

# =======================
#  Start DuckDB connection
# =======================
# Connect to an in-memory database or specify a file: database='imdb_merge.duckdb'
con = duckdb.connect(database=':memory:')

start_time = time.time()
print(" Starting IMDb data merge using DuckDB...\n")

# =======================
#  Stage progress tracker
# =======================
stages = [
    "Registering IMDb files",
    "Joining basics + ratings + crew + episode",
    "Joining akas",
    "Joining principals",
    "Joining name.basics",
    "Exporting to Parquet"
]
progress = tqdm(total=len(stages), desc="Progress", ncols=80, bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt}')

# =======================
#  Register TSV.GZ files as virtual tables
# =======================
# DuckDB can read directly from HTTPS URLs and handle compressed files.
# 'auto_detect=True' helps with schema, but we specify key params.
base_read_csv = "SELECT * FROM read_csv('{}', delim='\\t', nullstr='\\\\N', header=True, compression='gzip', auto_detect=True, parallel=True)"

con.execute(f"CREATE OR REPLACE VIEW name_basics AS {base_read_csv.format(URL_NAME_BASICS)};")
con.execute(f"CREATE OR REPLACE VIEW title_basics AS {base_read_csv.format(URL_TITLE_BASICS)};")
con.execute(f"CREATE OR REPLACE VIEW title_ratings AS {base_read_csv.format(URL_TITLE_RATINGS)};")
con.execute(f"CREATE OR REPLACE VIEW title_crew AS {base_read_csv.format(URL_TITLE_CREW)};")
con.execute(f"CREATE OR REPLACE VIEW title_episode AS {base_read_csv.format(URL_TITLE_EPISODE)};")
con.execute(f"CREATE OR REPLACE VIEW title_akas AS {base_read_csv.format(URL_TITLE_AKAS)};")
con.execute(f"CREATE OR REPLACE VIEW title_principals AS {base_read_csv.format(URL_TITLE_PRINC)};")

progress.update(1)
progress.set_description(stages[1])

# =======================
#  Perform joins step by step
# =======================
# This step-by-step materialization helps manage memory
con.execute("""
CREATE OR REPLACE TABLE merged_core AS
SELECT *
FROM title_basics b
LEFT JOIN title_ratings r USING (tconst)
LEFT JOIN title_crew c USING (tconst)
LEFT JOIN title_episode e USING (tconst);
""")
progress.update(1)
progress.set_description(stages[2])

con.execute("""
CREATE OR REPLACE TABLE merged_with_akas AS
SELECT *
FROM merged_core mc
LEFT JOIN title_akas a ON mc.tconst = a.titleId;
""")
progress.update(1)
progress.set_description(stages[3])

con.execute("""
CREATE OR REPLACE TABLE merged_with_principals AS
SELECT *
FROM merged_with_akas ma
LEFT JOIN title_principals p ON ma.tconst = p.tconst;
""")
progress.update(1)
progress.set_description(stages[4])

con.execute("""
CREATE OR REPLACE TABLE imdb_final AS
SELECT *
FROM merged_with_principals mp
LEFT JOIN name_basics n ON mp.nconst = n.nconst;
""")
progress.update(1)
progress.set_description(stages[5])

# =======================
#  Export to Parquet
# =======================
con.execute("""
COPY (SELECT * FROM imdb_final) 
TO 'imdb_merged_duckdb.parquet' (FORMAT PARQUET, COMPRESSION 'SNAPPY', ROW_GROUP_SIZE 100000);
""")
progress.update(1)
progress.close()

# =======================
#  Clean up
# =======================
con.close()
elapsed = (time.time() - start_time)
print(f"\n✅ Done! Merged dataset saved as imdb_merged_duckdb.parquet (Elapsed: {elapsed:.2f} seconds)")
print("   Reload it fast with: pd.read_parquet('imdb_merged_duckdb.parquet')")